In [1]:
import os
import json
from openai import OpenAI
from datetime import datetime, timedelta
from dotenv import load_dotenv
from core.messages_service import MessagesService
from core.json_service import dump_json_data

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
# Initialize conversation history
messages_service = MessagesService()
openai_model = "gpt-3.5-turbo"
user_interest_enum = None

# Initial system message
system_prompt = """You are an assistant that helps users find sushi restaurants and parking spots in Munich. 
When a user expresses interest in either sushi, parking, or both:
1. Use the set_user_interest function to record their preference
2. Provide relevant parking or restaurant information
3. ALWAYS respond with a natural language message summarizing the information for the user

Example response: 'Here are some parking spots near your location: Marienplatz, City, and Platzl Hotel München. Let me know if you need directions to any of these!'"""

messages_service.add_system_message(system_prompt)

user_prompt = "I want to park my car"
# user_prompt = input("eg. I am looking for a Sushi Restaurant in Munich.")
messages_service.add_user_message(user_prompt)

messages_service.print_messages()

{'role': 'system', 'content': "You are an assistant that helps users find sushi restaurants and parking spots in Munich. \nWhen a user expresses interest in either sushi, parking, or both:\n1. Use the set_user_interest function to record their preference\n2. Provide relevant parking or restaurant information\n3. ALWAYS respond with a natural language message summarizing the information for the user\n\nExample response: 'Here are some parking spots near your location: Marienplatz, City, and Platzl Hotel München. Let me know if you need directions to any of these!'"}
{'role': 'user', 'content': 'I want to park my car'}


In [3]:
from tools.user_interest_tools import user_interest_tools
from tools.parking_tools import parking_tools
from tools.sushi_tools import sushi_tools

In [4]:
tool_choice = {
    "type": "function",
    "function": {"name": "get_user_interest"}
}

In [5]:
# Get user interest
completion = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    tool_choice=tool_choice,
    tools=user_interest_tools,
)

# The json.loads function converts the string to a Python object

from enums.user_interest_enum import UserInterest
from services.user_interest_service import get_user_interest

message = completion.choices[0].message
print(message)
arguments = json.loads(message.tool_calls[0].function.arguments)

# Add the assistant's natural language response to the conversation
assistant_message = arguments.get("assistant_message")
if assistant_message:
    messages_service.add_assistant_message(assistant_message)
messages_service.print_last_message()


# Get the user's interest
interest = arguments.get("interest")
print(interest)
user_interest_enum = UserInterest(interest)

chosen_function = eval(message.tool_calls[0].function.name)
user_interest = chosen_function(interest)


user_interest_json = dump_json_data(user_interest)
print(user_interest_json)

messages_service.add_function_message(message.tool_calls[0].function.name, user_interest_json)
messages_service.print_last_message()

ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_AxxNuTW9d49UM8A7pzSno8Du', function=Function(arguments='{"interest":"PARKING","assistant_message":"Sure, I can help you find parking spots. Where are you located in Munich?"}', name='get_user_interest'), type='function')])
{'role': 'assistant', 'content': 'Sure, I can help you find parking spots. Where are you located in Munich?'}
PARKING
[{"parking": ["Marienplatz", "City", "Platzl Hotel München"]}]
{'role': 'function', 'name': 'get_user_interest', 'content': '[{"parking": ["Marienplatz", "City", "Platzl Hotel München"]}]'}


In [6]:
user_prompt = "What is the cheapest parking spot near me?"
messages_service.add_user_message(user_prompt)

In [7]:
from services.function_call_service import FunctionCallService
from services.parking_service import get_cheapest_spots

if user_interest_enum == UserInterest.PARKING:
    tools = parking_tools
elif user_interest_enum == UserInterest.SUSHI:
    tools = sushi_tools
else:
    tools = parking_tools + sushi_tools

completion = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    tools=tools,
    tool_choice="auto",
)
assistant_message = arguments.get("assistant_message")
if assistant_message:
    messages_service.add_assistant_message(assistant_message)
messages_service.print_last_message()

function_call_service = FunctionCallService(messages_service)


message = completion.choices[0].message
print(message)


chosen_function = eval(message.tool_calls[0].function.name)
user_interest = chosen_function()


user_interest_json = dump_json_data(user_interest)
print(user_interest_json)

messages_service.add_function_message(message.tool_calls[0].function.name, user_interest_json)
messages_service.print_last_message()



feedback_completion = client.chat.completions.create(
    model=openai_model,
    messages=messages_service.get_messages(),
    tools=tools,
)

message = feedback_completion.choices[0].message
print(message)



{'role': 'assistant', 'content': 'Sure, I can help you find parking spots. Where are you located in Munich?'}
ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_UyFTOBxffiYE72SREHVieejw', function=Function(arguments='{}', name='get_cheapest_spots'), type='function')])
["City", {"priceSummaryText": "6.00 € / 2h", "free": false}, {"listPrices": [{"service": "PARKING", "price": "Open 24 hours: 3.00 € / 1 hour"}, {"service": "PARKING", "price": "Open 24 hours: 6.00 € / 2 hours"}, {"service": "PARKING", "price": "Open 24 hours: 9.00 € / 3 hours"}, {"service": "PARKING", "price": "Open 24 hours: 12.00 € / 4 hours"}, {"service": "PARKING", "price": "Open 24 hours: 15.00 € / 5 hours"}, {"service": "PARKING", "price": "Open 24 hours: 18.00 € / 6 hours"}, {"service": "PARKING", "price": "Open 24 hours: 21.00 € / 7 hours"}, {"service": "PARKING", "price": "Open 24 hours: 24.00 € / 24 hours"}]}]
{'r

In [8]:
# reset jupiter kernel
%reset -f